# Notebook #1: Data Extraction
## Importing tabular data onto Rhino with SQL queries
In this notebook, you'll use SQL to query from an external database (such as a health system's clinical data warehouse) and import the results of those queries onto the Rhino Federated Computing Platform.

### Import the Rhino Health Python library
The code below imports various classes and functions from the `rhino_health` library, which is a custom library designed to interact with the Rhino Federated Computing Platform. More information about the SDK can be found on our [Official SDK Documentation](https://rhinohealth.github.io/rhino_sdk_docs/html/autoapi/index.html) and on our [PyPI Repository Page](https://pypi.org/project/rhino-health/) 

In [3]:
pip install --upgrade rhino_health

import getpass
from pprint import pprint
import rhino_health as rh
from rhino_health.lib.endpoints.dataset.dataset_dataclass import DatasetCreateInput
from rhino_health.lib.endpoints.project.project_dataclass import ProjectCreateInput
from rhino_health.lib.endpoints.sql_query.sql_query_dataclass import (SQLQueryImportInput,SQLQueryInput,SQLServerTypes,ConnectionDetails)
from rhino_health.lib.endpoints.code_object.code_object_dataclass import (CodeObject,CodeObjectCreateInput,CodeObjectRunInput,CodeTypes)

### Authenticate to the Rhino FCP
The `RhinoSession` class in the `rhino_health` library is a comprehensive interface for interacting with various endpoints in the Rhino Health ecosystem. It offers direct access to multiple specialized endpoints, including AI models, cohorts, data schemas, model results, projects, and workgroups, facilitating a wide range of operations in healthcare data management and analysis. The class also supports features like two-factor authentication and user switching, enhancing security and flexibility in handling different user sessions and workflows within the Rhino Health platform.

In [6]:
my_username = "daniel@rhinohealth.com" # Replace this with the email you use to log into Rhino Health
session = rh.login(username=my_username, password=getpass.getpass())

 ········


### Identify the desired project in the Rhino UI.
Before completing this step using the Python SDK, create a project on the Rhino web platform. Once the project has been created, copy the UID from the project you just created in the UI by navigating to the homepage, pressing on the three-vertical dot button in your project's square, and then selecting the button Copy UID.

![image info](images/copy_uid.png)

In [7]:
project_uid = 'c19cbf1a-b7d2-475d-a94c-c408aac77bef' # Replace with your Project's UID
workgroup_uid = session.project.get_collaborating_workgroups(project_uid)[0].uid

### Connection Setup
The `rhino_health.lib.endpoints.sql_query.sql_query_dataclass` module in the Rhino Health library provides classes to handle SQL queries against external databases and import data into the Rhino Federated Computing Platform. It includes `SQLQueryInput` for specifying parameters of a SQL query, `SQLQueryImportInput` for importing a cohort from an external SQL database query, and `SQLQuery`, a class representing an executed SQL query. More information about Rhino's SQL classes can be found by reviewing our SDK documentation [here](https://rhinohealth.github.io/rhino_sdk_docs/html/autoapi/rhino_health/lib/endpoints/sql_query/index.html).

**Please contact daniel@rhinohealth.com or yuval@rhinohealth.com for credentials to a test database that can be used for this notebook.**

In [8]:
sql_db_user = "rhino" # Replace this with your DB username (make sure the user has read-only permissions to the DB).
external_server_url = "ext-hospital-data.covi47dnmpiy.us-east-1.rds.amazonaws.com:5432" # Replace this with url + port of the SQL DB you want to query (ie "{url}:{port}").
db_name = "hospital_data" # Replace this with your DB name.

connection_details = ConnectionDetails(
    server_user=sql_db_user,
    password=getpass.getpass(),    
    server_type=SQLServerTypes.POSTGRESQL, # Replace POSTGRESQL with the relevant type of your sql server (See docs for all supported types).
    server_url=external_server_url,
    db_name=db_name
)

 ········


### Writing SQL queries against the DB
Using the `SQLQueryImportInput` function will allow us to query an external relational database and import the results of the query as a cohort. A Cohort is a central concept on the Rhino platform; to learn more, please navigate to this [link](https://docs.rhinohealth.com/hc/en-us/articles/12384748397213-What-is-a-Cohort-)

Executing the `SQLQueryImportInput` function requires a few arguments:
- cohort_name (str): Name for the cohort you are creating.
- is_data_deidentified (bool): Indicates if the data in the query is deidentified for privacy reasons.
- connection_details (ConnectionDetails): Details like URL, user, and password to connect to the SQL server.
- data_schema_uid (Optional[str]): The unique identifier for the data schema in the context of the query.
- timeout_seconds (int): Time limit in seconds for the query execution.
- project_uid (str): Unique identifier for the project context of the query.
- workgroup_uid (str): Unique identifier for the workgroup context of the query.
- sql_query (str): The actual SQL query to be run.

#### Table 1: Patient Admission Data
Our first query will retrieve patient demographics and associated clinical codes from inpatient admissions for patients with chest x-rays (see the WHERE clause, where we identify a selection of chest x-rays in the MIMIC v4 database).

In [22]:
site1_demographics_sql="""
SELECT distinct pat.subject_id, adm.hadm_id, pat.anchor_age + (EXTRACT(YEAR FROM adm.admittime) - pat.anchor_year) AS age,
    pat.gender, adm.insurance, adm.admission_type,adm.admission_location,adm.discharge_location,adm.language,
    adm.marital_status, adm.race, icd.icd_code as diagnosis_code,proc.icd_code as procedure_code
FROM mimiciv_hosp.admissions adm
LEFT JOIN mimiciv_hosp.patients pat ON pat.subject_id = adm.subject_id
LEFT JOIN mimiciv_hosp.diagnoses_icd icd ON adm.subject_id = icd.subject_id AND adm.hadm_id = icd.hadm_id
LEFT JOIN mimiciv_hosp.procedures_icd proc ON adm.subject_id = proc.subject_id AND adm.hadm_id = proc.hadm_id
LEFT JOIN mimiciv_cxr.study_list study ON adm.subject_id =study.subject_id
WHERE study.study_id in(57375967,50771383,54205396,53186264,54684191,50531538,59965534,51029426,50336039,51967845,53970869,54898709,57084339,58117097,58509443,
58555910,58733084,59289932,51449744,55590752,56617354,57652741,50230446,50252971,50702026,50789010,51737583,54855307,56421164,58996402,59565087,52358194,
51732447,53447201,54136122,58882809,54076811,53022275,58261299,57661212,52821744,53831730,56167317,57754443,50548939,55758528,58974095,51613820,58898689,55328702,
52654671,53468612,55928380,53461983,58400857,57874958,58971884,59558528,51497652,53161617,54277770,59484629,51212589,58414548,53818182,58812027,59585309,53534710,
58748017,53445324,58890389,53977911,57107380,58184428,53522120,55014265)
"""
import_run_params = SQLQueryImportInput(
    session = session,
    project = project_uid, # The project/workgroup will be used to validate permissions (including and k_anonymization value)
    workgroup = workgroup_uid,
    connection_details = connection_details,
    dataset_name = 'site1_demographics',
    data_schema_uid = None, # Auto-Generating the Output Data Schema for the Cohort
    timeout_seconds = 1200,
    is_data_deidentified = True,
    sql_query = site1_demographics_sql
)

response = session.sql_query.import_dataset_from_sql_query(import_run_params)

Waiting for SQL query to complete (0 hours 0 minutes and a second)
Waiting for SQL query to complete (0 hours 0 minutes and 11 seconds)
Run finished successfully


#### Table 2: EHR Observations
Our second query will retrieve observations from our clinical information system, including patient BMI, height, weight, and diastolic and systolic blood pressure.

In [23]:
site1_observations_sql = """
SELECT
   omr.subject_id,
   omr.chartdate,
   omr.result_name,
   max(omr.result_value) as result
FROM mimiciv_hosp.omr omr
LEFT JOIN mimiciv_cxr.study_list study
ON omr.subject_id =study.subject_id
WHERE study.study_id in (57375967,50771383,54205396,53186264,54684191,50531538,59965534,51029426,50336039,51967845,53970869,54898709,57084339,58117097,58509443,58555910,58733084,59289932,51449744,55590752,56617354,57652741,50230446,50252971,50702026,50789010,51737583,54855307,56421164,58996402,59565087,52358194,51732447,53447201,54136122,58882809,54076811,53022275,58261299,57661212,52821744,53831730,56167317,57754443,50548939,55758528,58974095,51613820,58898689,55328702,52654671,53468612,55928380,53461983,58400857,57874958,58971884,59558528,51497652,53161617,54277770,59484629,51212589,58414548,53818182,58812027,59585309,53534710,58748017,53445324,58890389,53977911,57107380,58184428,53522120,55014265)
GROUP BY omr.subject_id, omr.chartdate, omr.result_name
"""

import_run_params = SQLQueryImportInput(
    session = session,
    project = project_uid, # The project/workgroup will be used to validate permissions (including and k_anonymization value)
    workgroup = workgroup_uid,
    connection_details = connection_details,
    dataset_name = 'site1_outcome',
    data_schema_uid = None, # Auto-Generating the Output Data Schema for the Cohort
    timeout_seconds = 1200,
    is_data_deidentified = True,
    sql_query = site1_observations_sql
)

response = session.sql_query.import_dataset_from_sql_query(import_run_params)

Waiting for SQL query to complete (0 hours 0 minutes and a second)
Waiting for SQL query to complete (0 hours 0 minutes and 11 seconds)
Run finished successfully


### Importing chest x-rays from a PACS system into my Rhino client
Next, we'll import chest x-rays onto our Rhino client so that we can train our AI model. Chest x-rays that can be used for model training are included in this repository and are in DICOM format (see `/data/site1/dicom`). Associated patient metadata is also available (see `/data/site1/dicom/site1_dicom_metadata.csv`). 

If you are interested in learning more about how data can be imported from your local computing environment into the Rhino Federated Computing Platform, please refer to this section of our documentation [here](https://docs.rhinohealth.com/hc/en-us/articles/12385912890653-Adding-Data-to-your-Rhino-Federated-Computing-Platform-Client).

The data has been loaded in the `/rhino_data/image/dicom` path in the Rhino client. In addition, a file that provides metadata to associate the DICOM studies with the EHR data has been imported ('/rhino_data/image/metadata/aidev_cohort.csv').

In [21]:
dicom_path = "<insert path to `/data/site1/dicom` in your Rhino server>"
metadata_file = "<insert path to `/data/site1/site1_dicom_metadata.csv` in your Rhino server>"

dataset_creation_params = DatasetCreateInput(
    name="site1_images",
    description="Chest X-Rays",
    project_uid=project_uid, 
    workgroup_uid=workgroup_uid,
    data_schema_uid=None,
    image_filesystem_location=dicom_path,
    csv_filesystem_location = metadata_file,
    is_data_deidentified=True,
    method="filesystem"
)

import_run_params = SQLQueryImportInput(
    session = session,
    project = project_uid, # The project/workgroup will be used to validate permissions (including and k_anonymization value)
    workgroup = workgroup_uid,
    connection_details = connection_details,
    dataset_name = 'mimic_ehr_obs_dev',
    data_schema_uid = None, # Auto-Generating the Output Data Schema for the Cohort
    timeout_seconds = 1200,
    is_data_deidentified = True,
    sql_query = query_obs
)

ai_developer_image_dataset = session.dataset.add_dataset(dataset_creation_params)
print(f"Created new cohort '{ai_developer_image_dataset.name}' with uid '{ai_developer_image_dataset.uid}'")

Created new cohort 'mimic_cxr_dev' with uid 'd0502478-6ee1-452f-ad48-e5c1657bba96'


### What you'll see in the Rhino UI:
Once all three queries have been executed, you should see three cohorts in the user interface:
![image info](images/site1_datasets.png)